# Ficheux-like gate between a transmon and a fluxonium: scans of $t_{rise}$

In [ ]:
import numpy as np
import time
import qutip as qtp
import cmath
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pysqkit
from pysqkit.util.linalg import get_mat_elem, tensor_prod, hilbert_schmidt_prod
from pysqkit.util.phys import temperature_to_thermalenergy
from pysqkit.util.metrics import average_process_fidelity, \
    average_gate_fidelity
from pysqkit.util.hsbasis import weyl_by_index
import pysqkit.util.transformations as trf
from typing import Callable, List, Dict
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
import json

from IPython.display import display, Latex

#%matplotlib notebook

import adaptive

from pysqkit.util.phys import temperature_to_thermalenergy
from pysqkit.util.transformations import kraus_to_ptm
from IPython.display import display, Latex
from pysqkit.util.linalg import get_mat_elem, tensor_prod

adaptive.notebook_extension()

The gate is based on the following driven Hamiltonian of a fluxonium and a transmon capacitively coupled

$$H = H^{(0)}_T + H^{(0)}_F + V + H_{drive},$$

$$H^{(0)}_T =  \hbar \omega_T \hat{b}^{\dagger}\hat{b} + \hbar \frac{\delta_T}{2} \hat{b}^{\dagger} \hat{b}^{\dagger}\hat{b} \hat{b},$$

$$H^{(0)}_F = 4 E_{C,F} \cdot \hat{n}^2_F + \frac{1}{2}E_{L,F}\cdot\hat{\varphi}^2_F - E_{J,F}\cos\left(\hat{\varphi}_F - \phi_{ext,F}\right),$$

$$V = J_C \cdot \hat{n}_T \cdot \hat{n}_F,$$

$$\frac{H_{drive}}{h} = \left( \varepsilon_T \hat{n}_T + \varepsilon_F \hat{n}_F \right) \cos(2\pi f_d t).$$

The (approximate) charge operator for the transmon in terms of annihilation and creation operators reads

$$
\hat{n}_T = i \biggl( \frac{E_J}{32 |\delta_T |} \biggr)^{1/4} (b^{\dagger} - b),
$$
where 
$$
E_J = \hbar \biggl(\frac{\omega_T - \delta_T}{8 | \delta_T |} \biggr)^2.
$$


# Introducing the qubits and the coupled system

In [ ]:
temperature = 0.020 # K
thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
d_comp = 4

#Transmon
levels_t = 3
transm = pysqkit.qubits.SimpleTransmon(
    label='T', 
    max_freq=4.5, 
    anharm=-0.3,
    diel_loss_tan=0.75*1e-6, #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,    
    dim_hilbert=levels_t
)

#Fluxonium
levels_f = 5
ec_f = .973
el_f = .457
ej_f = 8.0
# The external flux in units of \phi_0 is set to 1/2 by defauls:
# double well configuration

flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=.973, 
    induct_energy=.457, 
    joseph_energy=8.0, 
    diel_loss_tan=7.1*1e-6, #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy
)
flx.diagonalize_basis(levels_f)

# We also add a drive on the fluxonium
flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cz_drive_f',
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

d_leak = levels_t*levels_f - d_comp

jc = 0.07
coupled_sys = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
state_labels = ['00', '01', '10', '11']
comp_states = []
for label in state_labels:
    comp_states.append(coupled_sys.state(label)[1])

In [ ]:
def generalized_rabi_frequency(
    levels: List['str'],
    eps: Dict,
    drive_frequency: float,
    system: pysqkit.systems.system.QubitSystem
):
    if len(levels) != 2:
        raise ValueError('The generalized rabi frequency is defined betwen two levels. '
                        'Please specify the desired two levels.')
        
    qubit_labels = system.labels
    drive_op = 0
    for label in qubit_labels:
        drive_op += eps[label]*system[label].charge_op()
    
    in_energy, in_state = system.state(levels[0])
    out_energy, out_state = system.state(levels[1])   
    big_omega_transition = np.abs(get_mat_elem(drive_op, in_state, out_state))
    
    energy_transition = np.abs(in_energy - out_energy)
    drive_detuning = energy_transition - drive_frequency
    
    return np.sqrt(big_omega_transition**2 + drive_detuning**2)

def delta(
    system: pysqkit.systems.system.QubitSystem
) -> float:
    delta_gate = (system.state('13')[0] - system.state('10')[0]) - \
    (system.state('03')[0] - system.state('00')[0])
    return delta_gate

def func_to_minimize(
    x0: np.ndarray,
    levels_first_transition: List['str'],
    levels_second_transition: List['str'],
    system: pysqkit.systems.system.QubitSystem,
    eps_ratio_dict: Dict    
) -> float:
    
    """
    Function to minimize in order to match the parameters in order to 
    implement a CZ gate up to single-qubit rotations. It returns the modulus
    of [rabi_second_transition - rabi_first_transition, 
    delta_gate - rabi_first_transition]/delta_gate
    x0 : np.ndarray([eta_reference, drive_freq]) represents the parameters to be 
         minimized.
    levels_first_transition : List with the labels of the first transition whose
       generalized Rabi frequency has to be matched
    levels_second_transition : List with the labels of the second transition whose
                               generalized Rabi frequency has to be matched
    system: coupled system we are analyzing
    eps_ratio_dict: dictionary whose keys are system.labels. The entries correspond
                    to the ratios between the corresponding qubit drive and the 
                    reference drive.     
    
    """
    
    qubit_labels = system.labels
    eps = {}
    for qubit in qubit_labels:
        eps[qubit] = x0[0]*eps_ratio_dict[qubit]
    rabi_first_transition = generalized_rabi_frequency(levels_first_transition, eps, x0[1], system)
    rabi_second_transition = generalized_rabi_frequency(levels_second_transition, eps, x0[1], system)
    delta_gate = delta(system)
    y = np.sqrt( (rabi_first_transition - rabi_second_transition)**2 + \
                (rabi_first_transition - delta_gate)**2)
    return np.abs(y/delta_gate)

In [ ]:
x0 = np.array([0.03, 7.15]) #initial guess
eps_ratios = {'T': 0.0, 'F':1.0}
args_to_pass = (['00', '03'], ['10', '13'], coupled_sys, eps_ratios) 

start = time.time()

minimization_result = minimize(func_to_minimize, x0, args=args_to_pass)

end = time.time()

display(Latex(r'$\mathrm{{Minimization \, time}} = {:.3f} \, s$'.format(end - start)))
display(Latex(r'$f_{{\mathrm{{min}}}} = {:.2e}$'.format(minimization_result['fun'])))

# Setting up the system

In [ ]:
eps_drive = minimization_result['x'][0]
freq_drive = minimization_result['x'][1]
rabi_period = 1/delta(coupled_sys)
t_rise_ref = 15 #16.0
t_tot = 60 # 60
pts_per_drive_period = 5

nb_points = int(t_tot*freq_drive*pts_per_drive_period)
tlist = np.linspace(0, t_tot, nb_points)

coupled_sys['F'].drives['cz_drive_f'].set_params(phase=0, time=tlist, amp=eps_drive, freq=freq_drive)

simu_opt = qtp.solver.Options()
simu_opt.atol = 1e-12
simu_opt.rtol = 1e-10

cz = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]])
cz_super = trf.kraus_to_super(cz, weyl_by_index)

In [ ]:
def single_qubit_corrections(
    sup_op: np.ndarray,
    hs_basis: Callable[[int, int], np.ndarray]
) -> np.ndarray:
    sigma_m1 = tensor_prod([np.array([[0.0, 0.0], [1.0, 0.0]]), np.array([[1.0, 0.0], [0.0, 0.0]])])
    sigma_m2 = tensor_prod([np.array([[1.0, 0.0], [0.0, 0.0]]), np.array([[0.0, 0.0], [1.0, 0.0]])])
    sigma_m1_vec = trf.mat_to_vec(sigma_m1, hs_basis)
    sigma_m2_vec = trf.mat_to_vec(sigma_m2, hs_basis)
    evolved_sigma_m1_vec = sup_op.dot(sigma_m1_vec)
    evolved_sigma_m2_vec = sup_op.dot(sigma_m2_vec)
    evolved_sigma_m1 = trf.vec_to_mat(evolved_sigma_m1_vec, hs_basis)
    evolved_sigma_m2 = trf.vec_to_mat(evolved_sigma_m2_vec, hs_basis)
    phi10 = cmath.phase(hilbert_schmidt_prod(sigma_m1, evolved_sigma_m1))
    phi01 = cmath.phase(hilbert_schmidt_prod(sigma_m2, evolved_sigma_m2))
    p_phi10 = np.array([[1, 0], [0, np.exp(-1j*phi10)]])
    p_phi01 = np.array([[1, 0], [0, np.exp(-1j*phi01)]])
    return tensor_prod([p_phi10, p_phi01])

def get_fidelity_leakage(
    t_rise: float
):
    coupled_sys['F'].drives['cz_drive_f'].set_params(rise_time=t_rise)
    
    
    env_syst = pysqkit.tomography.TomoEnv(system=coupled_sys, time=2*np.pi*tlist, options=simu_opt)
    n_process = 4 #seems optimal choice on my (Ale's) laptop
    sup_op = env_syst.to_super(comp_states, weyl_by_index, n_process)
    sq_corr = single_qubit_corrections(sup_op, weyl_by_index)
    sq_corr_sup = trf.kraus_to_super(sq_corr, weyl_by_index)
    total_sup_op = sq_corr_sup.dot(sup_op)
    l1 = env_syst.leakage(comp_states)
    
    f_gate = average_gate_fidelity(cz_super, total_sup_op, l1)
    return f_gate, l1
    
    

In [ ]:
start =  time.time()

f_gate, l1 = get_fidelity_leakage(t_rise_ref)

end = time.time()
display(Latex(r'$\mathrm{{Computation \, time}} = {:.3f} \, s$'.format(end - start)))

We obtain a gate fidelity

In [ ]:
display(Latex(r'$F_{{gate}} = $ {:.5f}'.format(np.real(f_gate))))

and leakage

In [ ]:
display(Latex(r'$L_1 = $ {:.5f}'.format(np.real(l1))))

# Adaptive scan

In [ ]:
from operator import itemgetter

def simulation(t_rise):
    f_gate, l1 = get_fidelity_leakage(t_rise)
    
    # In the actual simulaiton the code that would go here would look more like:
    # amp, freq = scan_params
    # system['fluxonium'].drives[0].set_params(amp=amp, freq=freq)
    # gate_op = sq.tomography(system, in_states, out_states) 
    
    # Where in_states is an array of the |00>, |01>, |10>, |11> obtained from system
    # out_states will be probably the full system state array
    
    # Then one does the analyis on the gate_op
    
    result = {}
    result['gate_infidelity'] = 1 - np.real(f_gate)
    result['leakage'] = np.real(l1)
    return result

t_rise_var = 0.3
t_rise_bounds = ((1 - t_rise_var)*t_rise_ref, (1 + t_rise_var)*t_rise_ref)

learner = adaptive.Learner1D(simulation, (t_rise_bounds))
data_saver = adaptive.DataSaver(learner, arg_picker=itemgetter('gate_infidelity'))

def adaptive_goal(learner):
    return learner.npoints > 1 #250 #estimated time with 500 points 12 hours
     #return learner.loss() < 0.001 or learner.npoints > 2000
runner = adaptive.Runner(data_saver, goal=adaptive_goal)

In [ ]:
runner.live_info()

In [ ]:
runner.live_plot(update_interval=0.1)

In [ ]:
data_list = list(data_saver.extra_data.items())
save = True
if save == True:
    with open("tmp/data_scan_t_rise_t_tot_" + 
              str(t_tot) + ".txt", "w") as fp:
        json.dump(data_list, fp)   